In [1]:
!pip install fpgrowth_py

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for fpgrowth_py from https://files.pythonhosted.org/packages/18/51/e0cc561ab4a1079c1fad85a43bfa1a95fbe1f4f27c866d12b4d79f95b1ac/fpgrowth_py-1.0.0-py3-none-any.whl.metadata


In [1]:
from fpgrowth_py.fpgrowth import *
from fpgrowth_py.utils import *

In [2]:
from csv import reader

def getFromFile(fname):
    itemSetList = []
    with open(fname, 'r') as csvfile:
        csvreader = reader(csvfile)
        for row in csvreader:
            # Chaque ligne dans le CSV est une transaction, on transforme en liste
            itemSetList.append(row)  # Chaque ligne est une liste d'articles
    
    # Calculer la fréquence de chaque item (facultatif selon l'algorithme)
    frequency = getFrequencyFromList(itemSetList)
    return itemSetList, frequency


# Lire les données depuis le fichier CSV
itemSetList, frequency = getFromFile('C:\\Users\\laetitia.monnier\\Documents\\Docs Léti\\Bassam\\data.csv')

print(itemSetList)
print(frequency)

[['8585', '948', '1125', '1289', '1544', '1651'], ['656', '783', '1017', '1273', '1318', '1429', '1516'], ['112', '1034'], ['647', '704', '929', '1040', '1094', '1182', '1194', '1214', '1250'], ['11', '89', '870', '1094', '1605', '1940']]
[1, 1, 1, 1, 1]


In [17]:
from collections import defaultdict, OrderedDict
from csv import reader
from itertools import chain, combinations
from optparse import OptionParser
from fpgrowth_py.utils import *

def fpgrowth(itemSetList, minSupRatio, minConf):
    frequency = getFrequencyFromList(itemSetList)
    minSup = len(itemSetList) * minSupRatio
    fpTree, headerTable = constructTree(itemSetList, frequency, minSup)
    if(fpTree == None):
        print('No frequent item set')
    else:
        freqItems = []
        mineTree(headerTable, minSup, set(), freqItems)
        rules = associationRule(freqItems, itemSetList, minConf)
        return freqItems, rules

def fpgrowthFromFile(fname, minSupRatio, minConf):
    itemSetList, frequency = getFromFile(fname)
    minSup = len(itemSetList) * minSupRatio
    fpTree, headerTable = constructTree(itemSetList, frequency, minSup)
    if(fpTree == None):
        print('No frequent item set')
    else:
        freqItems = []
        mineTree(headerTable, minSup, set(), freqItems)
        rules = associationRule(freqItems, itemSetList, minConf)
        return freqItems, rules



# if __name__ == "__main__":
#     optparser = OptionParser()
#     optparser.add_option('-f', '--data.csv',
#                          dest='inputFile',
#                          help='CSV filename',
#                          default=None)
#     optparser.add_option('-s', '--minSupport',
#                          dest='minSup',
#                          help='Min support (float)',
#                          default=0.5,
#                          type='float')
#     optparser.add_option('-c', '--minConfidence',
#                          dest='minConf',
#                          help='Min confidence (float)',
#                          default=0.5,
#                          type='float')

#     (options, args) = optparser.parse_args()

#     freqItemSet, rules = fpgrowthFromFile(options.inputFile, options.minSup, options.minConf)


fpgrowthFromFile('kaggle.csv', 0.25, 0.25)


([{'Tea Powder'},
  {'Lassi'},
  {'Panner'},
  {'Cheese'},
  {'Butter'},
  {'Sugar'},
  {'Sweet'},
  {'Bread'},
  {'Yougurt'},
  {'Coffee Powder'},
  {'Ghee'},
  {'Milk'}],
 [])

In [24]:
def constructTree(itemSetList, frequency, minSup):
    headerTable = defaultdict(int)
    # Counting frequency and create header table
    for idx, itemSet in enumerate(itemSetList):
        for item in itemSet:
            headerTable[item] += frequency[idx]

    # Deleting items below minSup
    headerTable = dict((item, sup) for item, sup in headerTable.items() if sup >= minSup)
    if(len(headerTable) == 0):
        return None, None

    # HeaderTable column [Item: [frequency, headNode]]
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    # Init Null head node
    fpTree = Node('Null', 1, None)
    # Update FP tree for each cleaned and sorted itemSet
    for idx, itemSet in enumerate(itemSetList):
        itemSet = [item for item in itemSet if item in headerTable]
        itemSet.sort(key=lambda item: headerTable[item][0], reverse=True)
        # Traverse from root to leaf, update tree with given item
        currentNode = fpTree
        for item in itemSet:
            currentNode = updateTree(item, currentNode, headerTable, frequency[idx])

    return fpTree, headerTable

def updateTree(item, treeNode, headerTable, frequency):
    if item in treeNode.children:
        # If the item already exists, increment the count
        treeNode.children[item].increment(frequency)
    else:
        # Create a new branch
        newItemNode = Node(item, frequency, treeNode)
        treeNode.children[item] = newItemNode
        # Link the new branch to header table
        updateHeaderTable(item, newItemNode, headerTable)

    return treeNode.children[item]

def updateHeaderTable(item, targetNode, headerTable):
    if(headerTable[item][1] == None):
        headerTable[item][1] = targetNode
    else:
        currentNode = headerTable[item][1]
        # Traverse to the last node then link it to the target
        while currentNode.next != None:
            currentNode = currentNode.next
        currentNode.next = targetNode
        
        
constructTree('kaggle.csv', 1, 0.25)

TypeError: 'int' object is not subscriptable